Model with adding additional capacity for sources Wind, Nuclear, Solar

In [6]:
from gurobipy import Model, GRB

# Time periods
T = [2025, 2030, 2035]

# Energy sources
G = ['Wind', 'Solar', 'Oil', 'Nuclear', 'Hydro', 'Natural Gas', 'Coal & Coke', 'Biomass & Geothermal']

# Emission factors for each energy source (MTCO2e/GWh) #Mahrukh
E_factor_data = {
    'Wind': [0, 0, 0], 
    'Solar': [0, 0, 0],
    'Oil': [0.00037, 0.00037, 0.00037],
    'Nuclear': [0, 0, 0],
    'Hydro': [0, 0, 0],
    'Natural Gas': [0.00037, 0.00037, 0.00037],
    'Coal & Coke': [0.00076, 0.00076, 0.00076],
    'Biomass & Geothermal': [0, 0, 0]  
}

# Electricity demand for each time period (GWh) 
D_data = {2025: 611806.94, 2030: 694406.68, 2035: 787841.11} #Angel

# Emission reduction  for each time period (#MTCO2e) 
Goals = {2025: 35.19, 2030: 28.89, 2035: 0}  #Mahrukh


# Fixed generation capacity for each time period (GWh)  #Angel 
Fixed_GEN_data = {
    'Wind': [64389.48],
    'Solar': [12184.92],
    'Oil': [1379.91],
    'Nuclear': [78631.37],
    'Hydro': [402575.9],
    'Natural Gas': [106529.07],
    'Coal & Coke': [8184.54],
    'Biomass & Geothermal': [8281.2]
}


# Cost data for each energy source (CAD per GWh) #Tashfeen
Cost_data = {
    'Wind': [80830, 80830, 80830],
    'Solar': [205500, 205500, 205500],
    'Oil': [61650, 61650, 61650],
    'Nuclear': [89050, 89050, 89050],
    'Hydro': [30140, 30140, 30140],
    'Natural Gas': [97270, 97270, 97270],
    'Coal & Coke': [83570, 83570, 83570],
    'Biomass & Geothermal': [82200, 82200, 82200]
}

# Capacity increase costs for each power plant type (CAD)#Tashfeen
Increase_Cost = {
    'Nuclear': [9.59e9, 9.59e9, 9.59e9],
    'Solar': [12.878e9, 12.878e9, 12.878e9],
    'Wind': [4.11e9, 4.11e9, 4.11e9]
}

# Capacity increase amounts for each power plant type (GWh)#Tashfeen
Capacity_Added = {
    'Nuclear': [48180, 48180, 48180],
    'Solar': [139809.6, 139809.6, 139809.6],
    'Wind': [43800, 43800, 43800]
}

# Create the model
m = Model("ImprovedCanadaEnergyModel")

# Decision Variables
# Generation for each energy source and time period
Gen = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Generation")

# Emissions for each energy source and time period
Emissions = m.addVars(G, T, vtype=GRB.CONTINUOUS, name="Emissions")

# Emission deviations for each time period
Deviation = m.addVars(T, vtype=GRB.CONTINUOUS, name="Deviation")

# Decision variables for capacity increase (only for Wind, Solar, Nuclear)
Capacity_Increase = m.addVars(['Wind', 'Solar', 'Nuclear'], [2030,2035], vtype=GRB.CONTINUOUS, name="Capacity_Increase")

# Binary variables for capacity increase decision
Capacity_Increase_Decision = m.addVars(['Wind', 'Solar', 'Nuclear'],T,vtype=GRB.BINARY, name="CapacityIncreaseDecision")
Capacity_Increase_Decision_2 = m.addVars(['Wind', 'Solar', 'Nuclear'],T, vtype=GRB.BINARY, name="CapacityIncreaseDecision2")

# Objective Functions
# Objective 1: Minimize emissions deviation
m.setObjectiveN(sum(Deviation[t] for t in T), index=0, priority=1, name="Minimize_Emissions")

# Objective 2: Minimize cost 
Total_Cost = (
    sum(Cost_data[g][T.index(t)] * Gen[g, t] for g in G for t in T) +
    sum(Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision[g, t] +Increase_Cost[g][T.index(t)] * Capacity_Increase_Decision_2[g, t] for g in ['Wind', 'Solar', 'Nuclear'] for t in T)  # Fixed opening cost
)
m.setObjectiveN(Total_Cost, index=1, priority=0, name="Minimize_Cost")

# Constraints
# Emission goals
m.addConstrs((sum(Emissions[g, t] for g in G) <= Goals[t] + Deviation[t] for t in T), "EmissionGoals")

# Emission calculations
m.addConstrs((Emissions[g, t] == E_factor_data[g][T.index(t)] * Gen[g, t] for g in G for t in T), "EmissionCalculation")

# Demand satisfaction
for t in T:
    m.addConstr(sum(Gen[g, t] for g in G) >= D_data[t], "DemandSatisfaction_" + str(t))

#fixed capacity limits for all sources in 2025
for g in G:
    m.addConstr(Gen[g, 2025] <= Fixed_GEN_data[g][T.index(2025)], f"FixedCapacityLimit_{g}_2025")

# Adjusted capacity limits for 2030
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider added capacity in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030], f"AdjustedCapacityLimit_{g}_2030")
    else:
        # For other sources, use fixed capacity limits in 2030
        m.addConstr(Gen[g, 2030] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2030")

# Adjusted capacity limits for 2035, considering the capacity from 2030 and additional increase in 2035
for g in G:
    if g in ['Wind', 'Solar', 'Nuclear']:
        # For Wind, Solar, and Nuclear, consider the total capacity (2030 increase + 2035 increase)
        total_capacity_2035 = Fixed_GEN_data[g][0] + Capacity_Increase[g, 2030] + Capacity_Increase[g, 2035]
        m.addConstr(Gen[g, 2035] <= total_capacity_2035, f"TotalAdjustedCapacityLimit_{g}_2035")
    else:
        # For other sources, use fixed capacity limits in 2035
        m.addConstr(Gen[g, 2035] <= Fixed_GEN_data[g][0], f"FixedCapacityLimit_{g}_2035")


# Capacity increase constraints
for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:
                m.addConstr(Capacity_Increase[g, t] <= Capacity_Increase_Decision[g, t] * Capacity_Added[g][T.index(t)] +
                Capacity_Increase_Decision_2[g, t] * Capacity_Added[g][T.index(t)] , f"CapacityIncreaseLink_{g}_{t}")



#constraint to make sure powerplant only added if existing capacity is not enough
utilization_threshold = 1  # Define a threshold for capacity utilization
M = 1e15  

for g in ['Wind', 'Solar', 'Nuclear']:
    for t in T[1:]:  
        existing_capacity_expr = Fixed_GEN_data[g][0] + sum(Capacity_Increase[g, year] for year in [2030,2035] if year < t)

        # Binary variable indicating if capacity increase is needed
        y_gt = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}")
        m.addConstr(Capacity_Increase_Decision[g, t] + M*y_gt >=1)
        m.addConstr(Gen[g, t] - (utilization_threshold * existing_capacity_expr) - M * (1-y_gt) <= 0, name=f"ThresholdCheck_{g}_{t}")
        
        # Binary variable indicating if second capacity increase is needed
        y_gt2 = m.addVar(vtype=GRB.BINARY, name=f"y_{g}_{t}_2")
        m.addConstr(Capacity_Increase_Decision_2[g, t] + M * y_gt2 >= 1)
        m.addConstr(Gen[g, t] - (utilization_threshold * (existing_capacity_expr + Capacity_Added[g][T.index(t)])) - M * (1 - y_gt2) <= 0, name=f"ThresholdCheck_{g}_{t}_2")



# Phase out non-renewables
m.addConstr(Gen['Coal & Coke', 2030] == 0, "Coal_PhaseOut_2035")
m.addConstr(Gen['Oil', 2035] == 0, "Oil_PhaseOut_2035")
m.addConstr(Gen['Coal & Coke', 2035] == 0, "Coal_PhaseOut_2035")


#Pratical constraints that might need to be implemented depending on stakeholder preferences

#no second wind in 2035 constraint (could be any source depending on what we want to phase out)
#m.addConstr(Capacity_Increase_Decision_2['Wind', 2035] == 0, f"NoPlant_Wind_2035")


#no first wind power plant in 2035 constraint (could be any source depending on what we want to phase out)
#m.addConstr(Capacity_Increase_Decision['Wind', 2035] == 0, f"NoPlant_Wind_2035")

#emmision deviation any year (constraint allowing us to control deviation thereby increasing or decreasing costs)
#m.addConstr(Deviation[2025] >= 3)

    
    
# Optimize the model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    for t in T:
        print(f"Year {t}:")

        # Initialize variables to store total costs and generations
        total_cost_year = 0
        total_emissions_year = 0
        total_generation_year = 0
        total_capacity_increase_cost_year = 0  # Total cost of adding capacity for the year
        total_generation_cost_year = 0  # Total cost of generating electricity for the year
        deviation_year = Deviation[t].x

        # Print generation, emissions, and cost for each energy source
        for g in G:
            generation = Gen[g, t].x
            emissions = Emissions[g, t].x
            cost = generation * Cost_data[g][T.index(t)]

            total_generation_year += generation
            total_emissions_year += emissions
            total_generation_cost_year += cost

            print(f"  Generation ({g}): {generation} GWh")
            print(f"  Emissions ({g}): {emissions} MTCO2e")
            print(f"  Generation Cost ({g}): {cost} CAD")

        # Check and print details for each power plant type opened
        for g in ['Wind', 'Solar', 'Nuclear']:
            # First power plant
            power_plant_opened = Capacity_Increase_Decision[g, t].x
            if power_plant_opened > 0.5:  # Power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No {g} Power Plant Opened in {t}")

            # Second power plant
            power_plant_opened_2 = Capacity_Increase_Decision_2[g, t].x
            if power_plant_opened_2 > 0.5:  # Second power plant opened in year t
                added_capacity = Capacity_Added[g][T.index(t)]
                cost_of_increase = Increase_Cost[g][T.index(t)]
                total_capacity_increase_cost_year += cost_of_increase

                print(f"  Second Power Plant Type Opened in {t}: {g}")
                print(f"  Added Capacity for {g} in {t}: {added_capacity} GWh")
                print(f"  Cost of Capacity Increase for {g} in {t}: {cost_of_increase} CAD")
            else:
                print(f"  No Second {g} Power Plant Opened in {t}")

        # Calculate and print total available capacity for Wind, Solar, and Nuclear
        for g in ['Wind', 'Solar', 'Nuclear']:
            total_capacity = Fixed_GEN_data[g][0]
            for year in T:
                if year <= t:
                    added_capacity = Capacity_Increase_Decision[g, year].x * Capacity_Added[g][T.index(year)]
                    added_capacity += Capacity_Increase_Decision_2[g, year].x * Capacity_Added[g][T.index(year)]
                    total_capacity += added_capacity
            print(f"  Total Available Capacity for {g} in {t}: {total_capacity} GWh")

        # Print total costs and generations
        total_cost_year = total_generation_cost_year + total_capacity_increase_cost_year
        print(f"Total Generation Cost for Year {t}: {total_generation_cost_year} CAD")
        print(f"Total Capacity Increase Cost for Year {t}: {total_capacity_increase_cost_year} CAD")
        print(f"Total Cost for Year {t}: {total_cost_year} CAD")
        print(f"Total Generation for Year {t}: {total_generation_year} GWh")
        print(f"Total Emissions for Year {t}: {total_emissions_year} MTCO2e")
        print(f"Emission Deviation for Year {t}: {deviation_year}")
else:
    print("No optimal solution found.")





Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 87 rows, 87 columns and 192 nonzeros
Model fingerprint: 0xeb0714b3
Variable types: 57 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [4e-04, 1e+15]
  Objective range  [1e+00, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 45 rows and 27 columns
Presolve time: 

In [7]:
for v in m.getVars():
    print(v.varName, v.x)

Generation[Wind,2025] 64389.48
Generation[Wind,2030] 151989.48
Generation[Wind,2035] 239589.48
Generation[Solar,2025] 0.0
Generation[Solar,2030] 0.0
Generation[Solar,2035] 10583.157297026977
Generation[Oil,2025] 1379.91
Generation[Oil,2030] 1379.91
Generation[Oil,2035] 0.0
Generation[Nuclear,2025] 78631.37
Generation[Nuclear,2030] 126811.37
Generation[Nuclear,2035] 126811.37
Generation[Hydro,2025] 402575.9
Generation[Hydro,2030] 402575.9
Generation[Hydro,2035] 402575.9
Generation[Natural Gas,2025] 48364.5399999999
Generation[Natural Gas,2030] 3368.819999999978
Generation[Natural Gas,2035] 0.002702972972972973
Generation[Coal & Coke,2025] 8184.54
Generation[Coal & Coke,2030] 0.0
Generation[Coal & Coke,2035] 0.0
Generation[Biomass & Geothermal,2025] 8281.2
Generation[Biomass & Geothermal,2030] 8281.2
Generation[Biomass & Geothermal,2035] 8281.2
Emissions[Wind,2025] 0.0
Emissions[Wind,2030] 0.0
Emissions[Wind,2035] 0.0
Emissions[Solar,2025] 0.0
Emissions[Solar,2030] 0.0
Emissions[Solar,20

Sensitivity Analysis

In [8]:
import pandas as pd

# Prepare lists to hold constraint information
constraint_names = []
constraint_sense = []
constraint_slack = []
constraint_rhs = []

# Iterate over the constraints in the model
for constr in m.getConstrs():
    # Append the data for each constraint to the lists
    constraint_names.append(constr.ConstrName)
    constraint_sense.append(constr.Sense)
    constraint_slack.append(constr.Slack)
    constraint_rhs.append(constr.RHS)

# Create a DataFrame using the data collected
constraints_df = pd.DataFrame({
    'Sense': constraint_sense,
    'Slack': constraint_slack,
    'RHS': constraint_rhs
},index=constraint_names)

# Display the DataFrame showing all the rows
pd.set_option('display.max_rows', None)
constraints_df


,Sense,Slack,RHS
EmissionGoals[2025],<,1.056430e+01,3.519000e+01
EmissionGoals[2030],<,2.713297e+01,2.889000e+01
EmissionGoals[2035],<,0.000000e+00,0.000000e+00
"EmissionCalculation[Wind,2025]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Wind,2030]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Wind,2035]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Solar,2025]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Solar,2030]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Solar,2035]",=,0.000000e+00,0.000000e+00
"EmissionCalculation[Oil,2025]",=,0.000000e+00,0.000000e+00


Model without adding additional capacity for sources Wind, Nuclear, Solar

In [9]:
from gurobipy import Model, GRB

# Time periods
T = [2025, 2030, 2035]

# Energy sources
G = ['Wind', 'Solar', 'Oil', 'Nuclear', 'Hydro', 'Natural Gas', 'Coal & Coke', 'Biomass & Geothermal']

# Emission factors for each energy source (MTCO2e/GWh)
E_factor_data = [
    [0, 0, 0],  # Wind
    [0, 0, 0],  # Solar
    [0.00037, 0.00037, 0.00037],  # Oil
    [0, 0, 0],  # Nuclear
    [0, 0, 0],  # Hydro
    [0.00037, 0.00037, 0.00037],  # Natural Gas
    [0.00076, 0.00076, 0.00076],  # Coal & Coke
    [0, 0, 0]  # Biomass & Geothermal
]

# Electricity demand for each time period (GWh)
D_data = [611806.94, 694406.68, 787841.11]

# Emission reduction goals for each time period (#MTCO2e)
Goals = [35.19, 28.89, 0]
#President Biden announced a target to achieve a 50-52 percent reduction from 2005 levels in economy-wide net greenhouse gas pollution by 2030

# Fixed generation capacity for each time period (GWh)



Fixed_GEN_data  = [
[64389.48,94244.94, 207475.59], # Wind
[12184.92,16278.47, 44913.70], # Solar
[1379.91,914.76, 726.09], # Oil
[78631.37,128015.91,164477.89], # Nuclear
[402575.9,447858.39, 467766.38],  # Hydro
[106529.07,128015.91, 130654.19],  # gas
[8184.54,656.14, 654.81], # Coal
[8281.2,9204,8, 19716.07] # Biomass
]

Cost_data = [
    [80830, 80830, 80830],  # wind
    [205500, 205500, 205500],  # solar
    [61650, 61650, 61650],  # oil
    [89050, 89050, 89050],  # nuclear
    [30140, 30140, 30140],  # hydro
    [97270, 97270, 97270],  # gas
    [83570, 83570, 83570], # coal
    [82200, 82200, 82200]# biomass
]

# Create the model
m = Model("ImprovedCanadaEnergyModel")

# Decision Variables
# Generation for each energy source and time period
Gen = m.addVars(len(G), len(T), vtype=GRB.CONTINUOUS, name="Generation")

# Emissions for each energy source and time period
Emissions = m.addVars(len(G), len(T), vtype=GRB.CONTINUOUS, name="Emissions")

# Emission deviations for each time period
Deviation = m.addVars(len(T), vtype=GRB.CONTINUOUS, name="Deviation")

# Objective Functions
# Objective 1: Minimize emissions deviation
m.setObjectiveN(sum(Deviation[t] for t in range(len(T))), index=0, priority=1, name="Minimize_Emissions")

# Objective 2: Minimize cost 
Total_Cost = (
    sum(Cost_data[g][t] * Gen[g, t] for g in range(len(G)) for t in range(len(T)))
    
)
m.setObjectiveN(Total_Cost, index=1, priority=0, name="Minimize_Cost")

# Constraints
# Emission goals
m.addConstrs((sum(Emissions[g, t] for g in range(len(G))) <= Goals[t] + Deviation[t] for t in range(len(T))), "EmissionGoals")

# Emission calculations
m.addConstrs((Emissions[g, t] == E_factor_data[g][t] * Gen[g, t] for g in range(len(G)) for t in range(len(T))), "EmissionCalculation")

# Demand satisfaction
m.addConstrs((sum(Gen[g, t] for g in range(len(G))) >= D_data[t] for t in range(len(T))), "DemandSatisfaction")

# Fixed capacity limits
m.addConstrs((Gen[g, t] <= Fixed_GEN_data[g][t] for g in range(len(G)) for t in range(len(T))), "FixedCapacityLimit")



# Phase out non-renewables like Coal and Oil by 2035 #Generation Target for 2035
m.addConstr(Gen[G.index('Oil'), T.index(2035)] == 0, "Oil_PhaseOut_2035")
m.addConstr(Gen[G.index('Coal & Coke'), T.index(2035)] == 0, "Coal_PhaseOut_2035")

# Constraint for full utilization of Wind, Solar, and Nuclear capacities in 2035 #Generation Target for 2035
m.addConstr(Gen[G.index('Wind'), T.index(2035)] == Fixed_GEN_data[G.index('Wind')][T.index(2035)], "Full_Use_Wind_2035")
m.addConstr(Gen[G.index('Solar'), T.index(2035)] == Fixed_GEN_data[G.index('Solar')][T.index(2035)], "Full_Use_Solar_2035")
m.addConstr(Gen[G.index('Nuclear'), T.index(2035)] == Fixed_GEN_data[G.index('Nuclear')][T.index(2035)], "Full_Use_Nuclear_2035")



# Optimize the model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    for t in T:
        print(f"Year {t}:")
        total_cost_year = 0
        total_emissions_year = 0
        total_generation_year = 0
        deviation_year = Deviation[T.index(t)].x  # Retrieve the deviation for the year

        for g in G:
            generation = Gen[G.index(g), T.index(t)].x
            emissions = Emissions[G.index(g), T.index(t)].x
            cost = generation * Cost_data[G.index(g)][T.index(t)]

            total_generation_year += generation
            total_emissions_year += emissions
            total_cost_year += cost

            print(f"  Generation ({g}): {generation} GWh")
            print(f"  Emissions ({g}): {emissions} MTCO2e")
            print(f"  Cost ({g}): {cost} USD")

        print(f"Total Generation for Year {t}: {total_generation_year} GWh")
        print(f"Total Emissions for Year {t}: {total_emissions_year} MTCO2e")
        print(f"Total Cost for Year {t}: {total_cost_year} USD")
        print(f"Emission Deviation for Year {t}: {deviation_year}")
else:
    print("No optimal solution found.")







Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 59 rows, 51 columns and 113 nonzeros
Model fingerprint: 0x56d2fd4f
Variable types: 51 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [1e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 8e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 53 rows and 29 columns
Presolved: 6 rows and 22 columns
---------------------------------------------------------------------------

Multi-objective

In [10]:
for v in m.getVars():
    print(v.varName, v.x)

Generation[0,0] 64389.48
Generation[0,1] 94244.94
Generation[0,2] 207475.59
Generation[1,0] 0.0
Generation[1,1] 0.0
Generation[1,2] 44913.7
Generation[2,0] 1379.91
Generation[2,1] 914.76
Generation[2,2] 0.0
Generation[3,0] 78631.37
Generation[3,1] 128015.91
Generation[3,2] 164477.89
Generation[4,0] 402575.9
Generation[4,1] 447858.39
Generation[4,2] 370973.93000000005
Generation[5,0] 48364.539999999906
Generation[5,1] 13512.539999999935
Generation[5,2] 0.0
Generation[6,0] 8184.54
Generation[6,1] 656.14
Generation[6,2] 0.0
Generation[7,0] 8281.2
Generation[7,1] 9204.0
Generation[7,2] 0.0
Emissions[0,0] 0.0
Emissions[0,1] 0.0
Emissions[0,2] 0.0
Emissions[1,0] 0.0
Emissions[1,1] 0.0
Emissions[1,2] 0.0
Emissions[2,0] 0.5105667
Emissions[2,1] 0.3384612
Emissions[2,2] 0.0
Emissions[3,0] 0.0
Emissions[3,1] 0.0
Emissions[3,2] 0.0
Emissions[4,0] 0.0
Emissions[4,1] 0.0
Emissions[4,2] 0.0
Emissions[5,0] 17.894879799999966
Emissions[5,1] 4.999639799999976
Emissions[5,2] 0.0
Emissions[6,0] 6.2202504